# Projeto 1 - Ciência dos Dados

### Alunos, digo, contratados da Apple:

#### Nome: Livia Brigido do Nascimento

#### Nome: Fabio Dotoli Ferreira Filho



Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

## INTRODUÇÃO: ## 
##### (Aqui vamos colocar algo sobre o projeto, quais serão os principais aprendizados...)

___
Carregando algumas bibliotecas que serão úteis:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI  #Essa serve para quê mesmo?
import re #utilizaremos para limpar o texto ('replace')
import nltk
from nltk.corpus import stopwords

In [2]:
# Confirmando o diretório a ser utilizado:

print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/fabiodff/Desktop/P1Cdados2021_2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Airpods.xlsx'

In [4]:
#A de treinamento (A que será responsável pela ação de "ensinar" a máquina o que é ou não é relevante)

train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Classificacao_Treino
0,alguem me da um apple watch 😭😭😭😭 um macbook pr...,0
1,@hschmaiske @matheus1lva airpods pro é bom? é....,1
2,mais uns airpods estragados… que sorte 🤦🏻‍♂️,1
3,"@annaeso @masbahnana isso eu sei, mas nem espe...",0
4,sonhando com o dia que vou achar airpods por m...,1


In [5]:
#Observando a quantidade relativa das duas classificações:

train.Classificacao_Treino.value_counts(True) #Precisamos equilibrar um pouco mais esses dados:

0    0.633333
1    0.366667
Name: Classificacao_Treino, dtype: float64

In [6]:
#A de teste (Com a classficação que fizemos apenas para comparar com a futura ação da máquina)

test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificacao_Teste
0,"favela venceu, o pai lançou um airpods monstro...",0
1,"[weverse]\n\n“namjoon oppa, se eles lançarem a...",0
2,@priboechat comprei um fone tipo airpods e amei,0
3,@_army_brasileira_7 \n _army_brasileira_7 \n\n...,0
4,"@abelardo_moura ainda bem que foi o de fio, po...",0


In [7]:
#Observando a quantidade relativa das duas classificações:

test.Classificacao_Teste.value_counts(True)

0    0.713333
1    0.286667
Name: Classificacao_Teste, dtype: float64

___
## Classificador automático de sentimento


### Produto escolhido ##

Os Airpods, lançamento da Apple no ano de 2016, são pequenos fones com conexão via bluetooth, que vêm acompanhados do estojo (usado para recarregá-los) e de um cabo lightning (carregador).

Os tweets considerados relevantes foram aqueles que citavam alguma caracteristica relacionada ao produto escolhido, como por exemplo a durabilidade, a facilidade em perder os fones, preço e críticas/elogios aos Airpods como um todo.

Os tweets irrelevantes, por sua vez, são aqueles que citam o produto, porém de uma forma que não é possivel extrair um feedback significativo sobre satisfação ou qualidades (positivas ou negativas) como, por exemplo, uma frase citando alguns itens aleatórios (dentre eles, os Airpods).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [67]:
#Montando a função de limpeza principal e para a limpeza de palavras mais específicas** (cleanup)

def cleanup_emoji(emoji1):
    return emoji1 in UNICODE_EMOJI

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '["\/!-.:?;"]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text) #substitui o primeiro argumento pelo segundo, e o terceiro argumento é onde será aplicada a substituição
    text_subbed = re.sub('\n', ' ', text_subbed) #retirando o enter (pular para a próxima linha)
    text_subbed = ''.join(' ' + char + ' ' if cleanup_emoji(char) else char for char in text_subbed).strip()

    
    padrao = r'https\w+'
    pattern2 = re.compile(padrao)
    text_subbed2 = re.sub(pattern2, '', text_subbed)
    
    return text_subbed2.lower()


def cleanup_links(text):
    
    padrao1 = r'@\w+' #remove o símbolo ou escrita e as letras após ele
    pattern1 = re.compile(padrao1)
    text_subbed = re.sub(pattern1, '', text)  
    
    padrao3 = r'#\w+'
    pattern3 = re.compile(padrao3)
    text_subbed3 = re.sub(pattern3, '', text_subbed)
    
    padrao4 = r'&\w+'
    pattern4 = re.compile(padrao4)
    text_subbed4 = re.sub(pattern4, '', text_subbed3)
    
    padrao5 = r'_\w+'
    pattern5 = re.compile(padrao5)
    text_subbed5 = re.sub(pattern5, '', text_subbed4)
    
    
    
    return text_subbed5



def stopwords_funcao(texto):
    stop_words = (nltk.corpus.stopwords.words('portuguese'))
    
    stopwords_lista = []
    
    for palavras in texto:
        if palavras not in stop_words: 
            stopwords_lista.append(palavras)
    
    return stopwords_lista

In [68]:
#Célula de Testes das funções:
variavel_teste="@joniel, a ou o que  gosta!?????? do: https:'hgah, a gente #skaska &sasjas"

variavel_limpa = cleanup_links(variavel_teste)
variavel_limpa
variavel_limpa2= cleanup(variavel_limpa)
variavel_limpa2

'a ou o que  gosta do  a gente'

In [69]:
#já foi dividido em rel. e irrel. pois depois não será possivel dividir por ser um df de apenas uma coluna 
train_relevantes_clean = train.loc[train["Classificacao_Treino"] == 1, 'Treinamento'].apply(cleanup_links).apply(cleanup)
train_relevantes_clean

1      airpods pro é bom é qualidade boa sim noise ca...
2      mais uns airpods estragados… que sorte  🤦  🏻 ‍...
4      sonhando com o dia que vou achar airpods por m...
5      tava pensando se esse airpods tinha valido o p...
7             oh trenzin q doi os zovido é esses airpods
                             ...                        
583           passando 3 meses encontrei os meus airpods
591    esqueci de carregar o airpods treinar sem músi...
593                                 perdi meus airpods 🥲
594    airpods compramos 2 um pra mim um pra minha es...
596    meu deus quem disse q ter airpods era uma boa ...
Name: Treinamento, Length: 220, dtype: object

In [70]:

#train_rel_clean.astype(str)
train_relevantes_lower = train_relevantes_clean.str.lower() #por que usar o .str ao inves de uma variavel-- Não quero trabalhar no dataset,mas em cada linha.
train_relevantes = train_relevantes_lower.str.split()
train_relevantes_clean = train_relevantes.apply(stopwords_funcao)


In [71]:
type(train_relevantes_clean)

pandas.core.series.Series

In [72]:
train_relevantes_l = []  #criando uma lista para englobar todas as outras listas criadas com o .split

for palavras in train_relevantes_clean:
    train_relevantes_l += palavras
serie_train_relevantes = pd.Series(train_relevantes_l)
serie_train_relevantes

0         airpods
1             pro
2             bom
3       qualidade
4             boa
          ...    
2201      airpods
2202          boa
2203        ideia
2204            q
2205      inferno
Length: 2206, dtype: object

In [73]:
serie_train_relevantes_absoluta = serie_train_relevantes.value_counts()
serie_train_relevantes_absoluta


airpods      222
pra           32
perdi         30
q             29
pro           25
            ... 
amava          1
deixar         1
kasdmksam      1
•              1
inears         1
Length: 1063, dtype: int64

In [74]:
train_irrelevantes_clean = train.loc[train["Classificacao_Treino"] == 0, 'Treinamento'].apply(cleanup_links).apply(cleanup)
train_irrelevantes_clean


0      alguem me da um apple watch  😭  😭  😭  😭  um ma...
3      isso eu sei mas nem esperaria algo assim é que...
6      sete anos atrás em 09092014 o valor do dólar e...
8         queria tanto um fone airpods mas zero dinheiro
9                                necessito de um airpods
                             ...                        
592    items do dreamcatcher para uma boa noite de so...
595                     vale a pena pegar um airpods pro
597                                 me compra um airpods
598    cara de mau mas no airpods tá tocando harry st...
599    algm tem link daquele airpods da shoppee q o i...
Name: Treinamento, Length: 380, dtype: object

In [75]:
train_irrelevantes_lower = train_irrelevantes_clean.str.lower() 
train_irrelevantes = train_irrelevantes_lower.str.split()
train_irrelevantes_clean = train_irrelevantes.apply(stopwords_funcao)

In [76]:
type(train_irrelevantes_clean)

pandas.core.series.Series

In [77]:
train_irrelevantes_clean

0      [alguem, apple, watch, 😭, 😭, 😭, 😭, macbook, pr...
3      [sei, esperaria, algo, assim, in, ficam, orelh...
6      [sete, anos, atrás, 09092014, valor, dólar, r2...
8         [queria, tanto, fone, airpods, zero, dinheiro]
9                                   [necessito, airpods]
                             ...                        
592    [items, dreamcatcher, boa, noite, sono, 🐼, cub...
595                    [vale, pena, pegar, airpods, pro]
597                                    [compra, airpods]
598     [cara, mau, airpods, tá, tocando, harry, styles]
599    [algm, link, daquele, airpods, shoppee, q, iph...
Name: Treinamento, Length: 380, dtype: object

In [78]:
train_irrelevantes_l = []  #criando uma lista para englobar todas as outras listas criadas com o .split

for palavras in train_irrelevantes_clean:
    train_irrelevantes_l += palavras
serie_train_irrelevantes = pd.Series(train_irrelevantes_l)
serie_train_irrelevantes

0          alguem
1           apple
2           watch
3               😭
4               😭
          ...    
4260      shoppee
4261            q
4262       iphone
4263    reconhece
4264     original
Length: 4265, dtype: object

In [79]:
serie_train_irrelevantes_absoluta = serie_train_irrelevantes.value_counts()
serie_train_irrelevantes_absoluta

airpods          390
pro               72
apple             62
iphone            59
comprar           54
                ... 
piano              1
provavelmente      1
🦋                  1
sofisticadas       1
☺                  1
Length: 1582, dtype: int64

In [80]:
test_relevantes_clean = test.loc[test["Classificacao_Teste"] == 1, 'Teste'].apply(cleanup_links).apply(cleanup)
test_relevantes_clean

14     dica do dia não coloque seus airpods na máquin...
16     olha se vc quiser algo que dure não compra air...
17     pqp achei que eu tinha perdido a capinha do ai...
24              esqueci de trazer meu airpods qero morre
34            o airpods é a melhor coisa q eu já comprei
                             ...                        
284    mano tu parea os airpods no iphone ele mostra ...
286    airpods duram umas 4 horas carregam em uns 30 ...
287          diz a pessoa que perdeu mais de 30 airpods 
289    eu preciiiiiso de um ipad pra assistir as aula...
298                  meus airpods tão com o volume baixo
Name: Teste, Length: 86, dtype: object

In [81]:
test_irrelevantes_clean = test.loc[test["Classificacao_Teste"] == 0, 'Teste'].apply(cleanup_links).apply(cleanup)
test_irrelevantes_clean

0         favela venceu o pai lançou um airpods monstro 
1      [weverse]  “namjoon oppa se eles lançarem airp...
2                    comprei um fone tipo airpods e amei
3      army  se você sair do airpods 3você não vive  ...
4      ainda bem que foi o de fio porque se fosse os ...
                             ...                        
294                    a meu mano eu tenho airpods certo
295    vou querer um macbook um ipad um airpods um ap...
296                       meu airpods até caiu da orelha
297    anonymous vazem o número do cartão do bolsonar...
299    sorte que comprei airpods pro de r1990 no shop...
Name: Teste, Length: 214, dtype: object

### Implementando a suavização de Laplace ###

In [82]:
train_tamanho_irrelevantes = len(serie_train_irrelevantes_absoluta)
train_tamanho_irrelevantes

1582

In [83]:
train_tamanho_relevantes = len(serie_train_relevantes_absoluta)
train_tamanho_relevantes

1063

In [84]:
Dimensao = train_tamanho_irrelevantes + train_tamanho_relevantes
D= Dimensao #Dimensão do nosso "universo" de palavras e emojis.

In [85]:
#Quantidade total de palavras em relevantes:
Nr= len(train_relevantes_l)
Nr

2206

In [86]:
#Quantidade total de palavras em irrelevantes:
Ni= len(train_irrelevantes_l)
Ni

4265

### Explicação das  três variáveis criadas: ###

## Enfim o classificador: ##


In [87]:
#Função que será executada com "apply":
def classificador_suavizado(tweet):
    prob_relevante=1
    prob_irrelevante=1
    alpha= 1
    lista_palavras = tweet.split()
    for palavra in lista_palavras:
        
        #Como vimos que, quando o termo não existia na lista, dava erro pois não se igualava à zero automaticamente:
        if palavra not in train_relevantes_l:
            serie_train_relevantes_absoluta[palavra]=0
        
        elif palavra not in train_irrelevantes_l:
            serie_train_irrelevantes_absoluta[palavra]=0
            
            
        #Não estava indo sem um outro if. Não sei o pq
        if palavra in train_relevantes_l and palavra in train_relevantes_l:
            prob_relevante*= (serie_train_relevantes_absoluta[palavra]+1)/ (Nr+D)
            prob_irrelevante*= (serie_train_irrelevantes_absoluta[palavra]+1)/ (Ni+D)
        
    #Em vez do que está abaixo, acrescentar em uma coluna nova a classficação feita pela maquina (Para comparar posteriormente com a nossa feita à mão)
    if prob_relevante>prob_irrelevante:
        return 1
    else:
        return 0


In [88]:
#Teste da função:
tweet="airpods não funciona"
print(classificador_suavizado(tweet))

1


In [89]:
testcopy= test.copy()
testcopy["classificacao_modelo"]= testcopy["Teste"].apply(classificador_suavizado)

#testcopy["classificacao_modelo"]="None"

#for linha, coluna in testcopy.iterrows():
    #testcopy.loc[linha, "classificacao_modelo"]=classificador_suavizado(testcopy.loc[linha, "Teste"])

In [90]:
testcopy["classificacao_modelo"].value_counts(True)

0    0.533333
1    0.466667
Name: classificacao_modelo, dtype: float64

In [91]:
testcopy

,Teste,Classificacao_Teste,classificacao_modelo
0,"favela venceu, o pai lançou um airpods monstro...",0,0
1,"[weverse]\n\n“namjoon oppa, se eles lançarem a...",0,0
2,@priboechat comprei um fone tipo airpods e amei,0,1
3,@_army_brasileira_7 \n _army_brasileira_7 \n\n...,0,0
4,"@abelardo_moura ainda bem que foi o de fio, po...",0,0
...,...,...,...
295,@jnkdress @anonymousbrof vou querer um macbook...,0,0
296,@lucxssl2 meu airpods até caiu da orelha,0,1
297,anonymous vazem o número do cartão do bolsonar...,0,0
298,meus airpods tão com o volume baixo :(,1,1


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [92]:
#Ver qual foi a distribuição do modelo
testcopy["classificacao_modelo"].value_counts(True)

0    0.533333
1    0.466667
Name: classificacao_modelo, dtype: float64

In [93]:
#Ver a distribuição que colocamos
testcopy["Classificacao_Teste"].value_counts(True)

0    0.713333
1    0.286667
Name: Classificacao_Teste, dtype: float64

### Porcentagem de erros e acertos:

In [94]:
#Matriz de confusão--Pesquisar
mat_porcentagem=pd.crosstab(testcopy.Classificacao_Teste, testcopy.classificacao_modelo, normalize=True)*100 #Multiplicar po 100 porque é porcentagem!
mat_porcentagem

classificacao_modelo,0,1
Classificacao_Teste,,
0,45.000000,26.333333
1,8.333333,20.333333


### Acurácia:

In [95]:
#Será aquelas em que ambos marcaram 0 e 1:
acuracia= mat_porcentagem[0][0]+mat_porcentagem[1][1]  #Coluna/ linha
acuracia

65.33333333333333

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

# • Diferentes cenários para a aplicação de Naïve Bayes: #

•Naive Bayes pode ser implementado quando há a necessidade de se diferenciar e-mails comuns, e-mails com assuntos voltados a business, e-mails voltados a notícias ou e-mails que são SPAM.

•Utilizar Naive Bayes para se ter um panorama do contágio ou não-contágio de pessoas expostas ao coronavírus , baseando-se na taxa de contágio do mesmo.
 
•Classificar a faixa etária de um público com base no vocabulário e abreviações utilizadas.

# • Por quê não se pode usar o classificador para gerar mais amostras?

• Não se pode usar o classificador para gerar mais amostras de treinamento, pelas mesmas serem feitas por humanos que tem uma interpretação do que significa ser relevante ou irrelevante, enquanto o classificador apenas considera as probabilidades das palavras nos tweets e no DataFrame. Então, quaisquer variações na forma de classificar geram erros (cometidos pelo próprio classificador) e usando o mesmo para gerar novas mensagens, os próprios erros são considerados corretos, tendendo cada vez mais para um resultado muito fora da curva.


# • Sugestão e implementação de melhorias no classificador:

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords

* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto

* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis

* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento

_____________________________________________________________________________________________________________________
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**